In [1]:
import pandas as pd
import numpy as np
import pymysql
import getpass

# Pandas - SQL, Joins, Groupbys

## 1. Connect to our MySQL database

In [3]:
pw = getpass.getpass()

········


In [5]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="ironhack",
                       passwd=getpass.getpass(),
                       db="olist")

········


In [6]:
conn

In [9]:
pd.read_sql("SELECT * FROM orders LIMIT 10", conn)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-16 13:22:11,2017-05-22 10:07:46,2017-05-26 12:55:51,2017-06-07
8,76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23 18:29:09,2017-01-25 02:50:47,2017-01-26 14:16:31,2017-02-02 14:08:10,2017-03-06
9,e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29 11:55:02,2017-07-29 12:05:32,2017-08-10 19:45:24,2017-08-16 17:14:30,2017-08-23


In [11]:
pd.read_sql("SELECT * FROM orders LIMIT 10", conn).dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

## 2. Rewrite SQL queries into pandas


#### (I) Joining `order_items` and `products`

```sql
SELECT
    oi.order_id,
    oi.order_item_id,
    oi.product_id,
    oi.price,
    p.product_category_name
FROM olist.order_items oi
    LEFT JOIN olist.products p
    ON oi.product_id = p.product_id
```

In [12]:
order_items = pd.read_sql("SELECT * FROM order_items", conn)
products = pd.read_sql("SELECT * FROM products", conn)

In [13]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [16]:
products.head()

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [17]:
order_items.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

In [18]:
products.dtypes

product_id                     object
product_category_name          object
product_name_length           float64
product_description_length    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [21]:
order_products = order_items.merge(right=products, 
                                   how="left", 
                                   on="product_id")[["order_id",
                                                     "order_item_id",
                                                     "product_id",
                                                     "price",
                                                     "product_category_name"]]

In [22]:
order_products

,order_id,order_item_id,product_id,price,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,moveis_decoracao
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,perfumaria
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,ferramentas_jardim
...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,299.99,utilidades_domesticas
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,350.00,informatica_acessorios
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,99.90,esporte_lazer
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,55.99,informatica_acessorios


#### (II) Add a GROUP BY

```sql
SELECT
    p.product_category_name,
    COUNT(*)                   AS items_sold,
    SUM(oi.price)              AS revenue
FROM olist.order_items oi
    LEFT JOIN olist.products p
    ON oi.product_id = p.product_id
GROUP BY
    p.product_category_name
```

In [27]:
type(order_products.groupby("product_category_name"))

pandas.core.groupby.generic.DataFrameGroupBy

In [37]:
per_category = (order_products
                .groupby("product_category_name")
                .agg({"price": "sum",
                      "order_id": "count"}))

Alternative ways to pass aggregate functions and groupbys:

In [30]:
order_products.groupby("product_category_name").sum()[["price"]]

,price
product_category_name,
agro_industria_e_comercio,72530.47
alimentos,29393.41
alimentos_bebidas,15179.48
artes,24202.64
artes_e_artesanato,1814.01
...,...
sinalizacao_e_seguranca,21509.23
tablets_impressao_imagem,7528.41
telefonia,323667.53


In [32]:
order_products.groupby("product_category_name").count()

,order_id,order_item_id,product_id,price
product_category_name,,,,
agro_industria_e_comercio,212,212,212,212
alimentos,510,510,510,510
alimentos_bebidas,278,278,278,278
artes,209,209,209,209
artes_e_artesanato,24,24,24,24
...,...,...,...,...
sinalizacao_e_seguranca,199,199,199,199
tablets_impressao_imagem,83,83,83,83
telefonia,4545,4545,4545,4545


In [35]:
(order_products
 .groupby("product_category_name")
 .agg({"price": np.sum,
       "order_id": np.size}))

,price,order_id
product_category_name,,
agro_industria_e_comercio,72530.47,212
alimentos,29393.41,510
alimentos_bebidas,15179.48,278
artes,24202.64,209
artes_e_artesanato,1814.01,24
...,...,...
sinalizacao_e_seguranca,21509.23,199
tablets_impressao_imagem,7528.41,83
telefonia,323667.53,4545


In [38]:
per_category

,price,order_id
product_category_name,,
agro_industria_e_comercio,72530.47,212
alimentos,29393.41,510
alimentos_bebidas,15179.48,278
artes,24202.64,209
artes_e_artesanato,1814.01,24
...,...,...
sinalizacao_e_seguranca,21509.23,199
tablets_impressao_imagem,7528.41,83
telefonia,323667.53,4545


In [40]:
per_category.loc["artes",]

price       24202.64
order_id      209.00
Name: artes, dtype: float64

In [41]:
per_category.columns

Index(['price', 'order_id'], dtype='object')

In [42]:
per_category.loc["artes","price"]

24202.639999999992

In [43]:
per_category.reset_index()

,product_category_name,price,order_id
0,agro_industria_e_comercio,72530.47,212
1,alimentos,29393.41,510
2,alimentos_bebidas,15179.48,278
3,artes,24202.64,209
4,artes_e_artesanato,1814.01,24
...,...,...,...
68,sinalizacao_e_seguranca,21509.23,199
69,tablets_impressao_imagem,7528.41,83
70,telefonia,323667.53,4545
71,telefonia_fixa,59583.00,264


Let's take all the steps and put them together:

```sql
SELECT
    p.product_category_name,
    COUNT(*)                   AS items_sold,
    SUM(oi.price)              AS revenue
FROM olist.order_items oi
    LEFT JOIN olist.products p
    ON oi.product_id = p.product_id
GROUP BY
    p.product_category_name
```

In [45]:
(order_items
 .merge(products, how="left", on="product_id")
 .groupby("product_category_name")
 .agg({"price": "sum",
       "order_id": "count"})
 .reset_index())

,product_category_name,price,order_id
0,agro_industria_e_comercio,72530.47,212
1,alimentos,29393.41,510
2,alimentos_bebidas,15179.48,278
3,artes,24202.64,209
4,artes_e_artesanato,1814.01,24
...,...,...,...
68,sinalizacao_e_seguranca,21509.23,199
69,tablets_impressao_imagem,7528.41,83
70,telefonia,323667.53,4545
71,telefonia_fixa,59583.00,264


In [50]:
# rename columns after agg
# add a column javier with hard-coded values

fav_df = (order_items
          .merge(products, how="left", on="product_id")
          .groupby("product_category_name")
          .agg({"price": "sum",
                "order_id": "count"})
          .reset_index()
          .rename(columns={"price": "revenue",
                           "order_id": "items_sold"})
          .assign(javier=1)
          .assign(bita=2))

,product_category_name,revenue,items_sold,javier,bita
0,agro_industria_e_comercio,72530.47,212,1,2
1,alimentos,29393.41,510,1,2
2,alimentos_bebidas,15179.48,278,1,2
3,artes,24202.64,209,1,2
4,artes_e_artesanato,1814.01,24,1,2
...,...,...,...,...,...
68,sinalizacao_e_seguranca,21509.23,199,1,2
69,tablets_impressao_imagem,7528.41,83,1,2
70,telefonia,323667.53,4545,1,2
71,telefonia_fixa,59583.00,264,1,2


In [53]:
# sorting

(order_items
 .merge(products, how="left", on="product_id")
 .groupby("product_category_name")
 .agg({"price": "sum",
       "order_id": "count"})
 .reset_index()
 .rename(columns={"price": "revenue",
                  "order_id": "items_sold"})
 .sort_values("revenue", ascending=False)).head(10)

,product_category_name,revenue,items_sold
11,beleza_saude,1258681.34,9670
66,relogios_presentes,1205005.68,5991
13,cama_mesa_banho,1036988.68,11115
32,esporte_lazer,988048.97,8641
44,informatica_acessorios,911954.32,7827
54,moveis_decoracao,729762.49,8334
26,cool_stuff,635290.85,3796
72,utilidades_domesticas,632248.66,6964
8,automotivo,592720.11,4235
40,ferramentas_jardim,485256.46,4347


### Column transformations: Decoding

In [69]:
ex_df = order_items[["price"]].copy()

In [70]:
ex_df = ex_df.assign(price_category=pd.NA)
ex_df

,price,price_category
0,58.90,NaN
1,239.90,NaN
2,199.00,NaN
3,12.99,NaN
4,199.90,NaN
...,...,...
112645,299.99,NaN
112646,350.00,NaN
112647,99.90,NaN
112648,55.99,NaN


In [72]:
ex_df.loc[ex_df["price"].le(100), "price_category"] = "cheap"

In [75]:
ex_df.loc[ex_df["price"].le(100), "price"]

0         58.90
3         12.99
5         21.90
6         19.90
9         53.99
          ...  
112642    55.00
112643    55.00
112647    99.90
112648    55.99
112649    43.00
Name: price, Length: 72337, dtype: float64

In [76]:
ex_df.loc[ex_df["price"].between(100,249.99,inclusive=False), "price_category"] = "medium"

In [81]:
ex_df["price_category"] = ex_df["price_category"].fillna("expensive")

In [82]:
ex_df

,price,price_category
0,58.90,cheap
1,239.90,medium
2,199.00,medium
3,12.99,cheap
4,199.90,medium
...,...,...
112645,299.99,expensive
112646,350.00,expensive
112647,99.90,cheap
112648,55.99,cheap


In [85]:
order_items = order_items.assign(price_category=ex_df["price_category"])

In [86]:
order_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,price_category
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cheap
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,medium
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,medium
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,cheap
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,medium
...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,expensive
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,expensive
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,cheap
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,cheap


In [89]:
order_items["price_category"].value_counts()

cheap        72337
medium       30554
expensive     9759
Name: price_category, dtype: int64

In [88]:
order_items["price_category"].value_counts(normalize=True)

cheap        0.642139
medium       0.271229
expensive    0.086631
Name: price_category, dtype: float64

Alternatively: Use `np.where()`

In [94]:
order_items.assign(alt_price_category=np.where(order_items["price"].le(100),
                                               "cheap",
                                               np.where(order_items["price"].le(250),
                                                        "medium",
                                                        "expensive")))

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,price_category,alt_price_category
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cheap,cheap
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,medium,medium
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,medium,medium
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,cheap,cheap
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,medium,medium
...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,expensive,expensive
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,expensive,expensive
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,cheap,cheap
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,cheap,cheap


```sql
WITH reviews_to_keep AS (
    SELECT
        order_id,
        MAX(review_answer_timestamp) AS latest_review_timestamp
    FROM order_reviews
    GROUP BY order_id),

latest_review_score AS (
    SELECT
        ore.order_id,
        ore.review_score
    FROM order_reviews ore
        INNER JOIN reviews_to_keep rtk
        ON ore.order_id = rtk.order_id
        AND ore.review_answer_timestamp = rtk.latest_review_timestamp),  -- for each order_id, the latest review rating

-- add previous pre-filter steps
seller_order_links AS (
    SELECT DISTINCT
        seller_id,
        order_id
    FROM order_items),

seller_orders_relations AS (
    SELECT
        s.seller_state,
        s.seller_id,
        sol.order_id
    FROM sellers s
        INNER JOIN seller_order_links sol
        ON s.seller_id = sol.seller_id)       -- for each seller, all order_ids they were associated with and their state

SELECT
    sor.seller_state,
    AVG(lrs.review_score)   AS avg_review_score,
    COUNT(1)                AS no_of_reviews
FROM seller_orders_relations sor
    INNER JOIN latest_review_score lrs
    ON sor.order_id = lrs.order_id
GROUP BY 1
ORDER BY 2 DESC;
```

---
Break it down into smaller components. Start with:

```sql
WITH reviews_to_keep AS (
    SELECT
        order_id,
        MAX(review_answer_timestamp) AS latest_review_timestamp
    FROM order_reviews
    GROUP BY order_id),

latest_review_score AS (
    SELECT
        ore.order_id,
        ore.review_score
    FROM order_reviews ore
        INNER JOIN reviews_to_keep rtk
        ON ore.order_id = rtk.order_id
        AND ore.review_answer_timestamp = rtk.latest_review_timestamp),
```


In [95]:
order_reviews = pd.read_sql("SELECT * FROM order_reviews;", conn)

In [96]:
order_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53


In [103]:
reviews_to_keep = (order_reviews
                   .groupby("order_id")
                   .agg({"review_answer_timestamp": "max"})
                   .rename(columns={"review_answer_timestamp": "latest_review_timestamp"})
                   .reset_index())

In [111]:
reviews_to_keep

,order_id,latest_review_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,2017-09-22 10:57:03
1,00018f77f2f0320c557190d7a144bdd3,2017-05-15 11:34:13
2,000229ec398224ef6ca0657da4fc703e,2018-01-23 16:06:31
3,00024acbcdf0a6daa1e931b038114c75,2018-08-15 16:39:01
4,00042b26cf59d7ce69dfabb4e55b4fd9,2017-03-03 10:54:59
...,...,...
99436,fffc94f6ce00a00581880bf54a75a037,2018-05-14 12:53:47
99437,fffcd46ef2263f404302a634eb57f7eb,2018-07-25 09:25:29
99438,fffce4705a9662cd70adb13d4a31832d,2017-10-29 21:33:52
99439,fffe18544ffabc95dfada21779c9644f,2017-08-18 12:24:05


In [112]:
latest_review_score = order_reviews.merge(reviews_to_keep, 
                                          left_on=["order_id", "review_answer_timestamp"],
                                          right_on=["order_id", "latest_review_timestamp"])

```sql
seller_order_links AS (
    SELECT DISTINCT
        seller_id,
        order_id
    FROM order_items),

seller_orders_relations AS (
    SELECT
        s.seller_state,
        s.seller_id,
        sol.order_id
    FROM sellers s
        INNER JOIN seller_order_links sol
        ON s.seller_id = sol.seller_id)   
```

In [120]:
order_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,price_category
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cheap
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,medium
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,medium
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,cheap
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,medium
...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,expensive
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,expensive
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,cheap
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,cheap


In [121]:
seller_order_links = order_items[["seller_id", "order_id"]].drop_duplicates()

In [122]:
sellers = pd.read_sql("SELECT * FROM sellers", conn)

In [124]:
seller_order_relations = sellers.merge(seller_order_links, on="seller_id")[["seller_state",
                                                                            "seller_id",
                                                                            "order_id"]]

```sql
SELECT
    sor.seller_state,
    AVG(lrs.review_score)   AS avg_review_score,
    COUNT(1)                AS no_of_reviews
FROM seller_orders_relations sor
    INNER JOIN latest_review_score lrs
    ON sor.order_id = lrs.order_id
GROUP BY 1
ORDER BY 2 DESC;
```

In [130]:
(seller_order_relations
 .merge(latest_review_score, on="order_id")
 .groupby("seller_state")
 .agg({"review_score": "mean",
       "order_id": "count"})
 .reset_index()
 .rename(columns={"review_score": "avg_review_score",
                  "order_id": "no_of_reviews"})
 .sort_values("avg_review_score", ascending=False))


,seller_state,avg_review_score,no_of_reviews
11,PA,4.500000,8
9,MS,4.387755,49
6,GO,4.289417,463
17,RN,4.274510,51
19,RS,4.271723,1991
13,PE,4.221675,406
10,MT,4.182482,137
8,MG,4.148577,7942
3,CE,4.142857,91
16,RJ,4.132951,4355


In [135]:
# using numpy agg functions
(seller_order_relations
 .merge(latest_review_score, on="order_id")
 .groupby("seller_state")
 .agg({"review_score": np.mean,
       "order_id": np.size})
 .reset_index()
 .rename(columns={"review_score": "avg_review_score",
                  "order_id": "no_of_reviews"})
 .sort_values("avg_review_score", ascending=False))


,seller_state,avg_review_score,no_of_reviews
11,PA,4.500000,8
9,MS,4.387755,49
6,GO,4.289417,463
17,RN,4.274510,51
19,RS,4.271723,1991
13,PE,4.221675,406
10,MT,4.182482,137
8,MG,4.148577,7942
3,CE,4.142857,91
16,RJ,4.132951,4355
